<h1> Overlapping Photons </h1>
Date: 23rd March 2023

Now let's investigate overlapping photons. First I need to do some event selection and prep the data.

In [1]:
# own skrips
import myfunctions as mf
#import helperfile as hf
# generalls libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import uproot
from tensorflow.keras.optimizers import Adam
import time
from scipy.optimize import curve_fit

In [2]:
from matplotlib.colors import LogNorm
from scipy import ndimage

As I need to try out some stuff, working with classes is not handy, as I need to load the data everytime again, so work with functions only first.

In [3]:
# load data
rootfile = uproot.open("./stage4_clusters.root")
event = rootfile["user202302;1"]
xMC = event["x_MC"].array(library="np") #[:40000]
yMC = event["y_MC"].array(library="np") #[:40000]
EMC = event["E_MC"].array(library="np") #[:40000]
x_truth = event["x_truth"].array(library="np") #[:40000]
y_truth = event["y_truth"].array(library="np") #[:40000]
E_truth = event["E_truth"].array(library="np") #[:40000]
x_fit = event["x_fit"].array(library="np") #[:40000]
y_fit = event["y_fit"].array(library="np") #[:40000]
E_fit = event["E_fit"].array(library="np") #[:40000]

In [4]:
clustersNxN, coord, ind_del = mf.form_cluster(xMC, yMC, EMC, clustershape=(5,5))
print("cut ", len(ind_del), " clusters")

This took  141.64326357841492 s
cut  146237  clusters


In [5]:
clustersNxN_2, coord_2, ind_del_2 = mf.form_cluster(xMC, yMC, EMC, clustershape=(6,6))
print("cut ", len(ind_del_2), " clusters")

This took  146.82255172729492 s
cut  48408  clusters


In [6]:
clustersNxN_3, coord_3, ind_del_3 = mf.form_cluster(xMC, yMC, EMC, clustershape=(7,7))
print("cut ", len(ind_del_3), " clusters")

This took  148.04006791114807 s
cut  6616  clusters


In [7]:
clustersNxN_4, coord_4, ind_del_4 = mf.form_cluster(xMC, yMC, EMC, clustershape=(8,8))
print("cut ", len(ind_del_4), " clusters")

This took  148.76361179351807 s
cut  105  clusters


So a (8,8) grid is beneficial!

But the problem is that I also need to check if the clusters overlapp... overwise this would not be an extra case!

**Update:** Well, they are close enough to not differ between the cases in a 7x7 grid.

In [8]:
def fill_zeros_random(cluster, ex, ey, cshape):
    '''make clusters to random NxN shape with zeros'''
    # csys = array with [x, y] at lower left corner, exactly in the center of the ecal cell!
    csys = np.array([ex[0], ey[0]])
    # cover case if only one cell in one dimension... np.histogram2D used +/- 0.5 for the binning then
    if len(ex) == 2:
        csys[0] = ex[0]+0.5
    if len(ey) == 2:
        csys[1] == ey[0] + 0.5
    
    cellsize = 3.83
    # cut clusters that are bigger than cshape! -> use bool return variable
    ret = True
    if(cluster.shape[0]>cshape[0] or cluster.shape[1]>cshape[1]):
        #print("Clusters are bigger than 5 celles: ", cluster.shape[0], cluster.shape[1])
        ret = False
        cluster_new = cluster
    else:
        dely = cshape[0] - cluster.shape[0]
        delx = cshape[1] - cluster.shape[1]
        # randomly choose how much upper left corner of cluster should be moved in NxN zero cluster 
        x = int(np.random.choice(np.arange(0, delx+1), 1))
        y = int(np.random.choice(np.arange(0, dely+1), 1))
        cluster_new = np.zeros(cshape[0]*cshape[1]).reshape(cshape) # create empty/zero cluster
        cluster_new[y:(cluster.shape[0]+y), x:(cluster.shape[1]+x)] = cluster # insert ecal cluster into the zero cluster
        # new coordinate system
        csys[0] = csys[0] - x*cellsize
        csys[1] = csys[1] - (cshape[0] - cluster.shape[0] - y)*cellsize # as one also needs to shift from upper left to lower left corner
       
    return cluster_new, csys, ret

In [9]:
def is_connected(matrix): # FUNKTIONIERT NICHT
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.find_objects.html#scipy.ndimage.find_objects 

    # find connected components in the matrix

    labeled_array, num_features = ndimage.label(matrix)
    struct = ndimage.generate_binary_structure(2,2)

    # dilate the labeled array with the kernel

    dilated = ndimage.binary_dilation(labeled_array.astype(bool), structure=struct).astype(np.uint8)

    # check if there is only one non-zero feature

    if num_features != 1: 
        return True
    return len(np.unique(dilated)) == 2

In [10]:
def check_overlapp(cluster, ex, ey):
    yes = True
    '''
    if is_connected(cluster) == False:
        plt.imshow(cluster, norm=LogNorm())
        plt.colorbar()
        plt.show()
        print("Zellen in x: ", len(ex)-1)
        print("Zellen in y: ", len(ey)-1)
        print("ChatGPT: ", is_connected(cluster))
    '''
    return yes

In [11]:
def form_cluster(xMC, yMC, EMC, clustershape=(5,5)):
    '''make nxn shape from phast data - return cluster, coordinate points and edges of histogram'''
    t1 = time.time()
    arr_cluster = np.zeros((len(xMC), clustershape[0], clustershape[1])) # space holder of clusters 
    c_sys = np.zeros((len(xMC), 2)) # space holder for coordinate system
    ind_i = [] # add all indecies of not used clusters
    num_notoverlapp = 0
    for i in range(len(xMC)):
        x = xMC[i]
        y = yMC[i]
        E = EMC[i]
        cellsize = 3.83
        binx = round((x.max() - x.min())/ cellsize) +1
        biny = round((y.max() - y.min()) / cellsize) +1
        histo, ex, ey = np.histogram2d(x, y, bins=[binx,biny], weights=E, density=False)
        cluster = np.flip(histo.T, axis=0) # make shape more intuitive - looks like scattered x,y now when array is printed!
        # hier ueberpruefen ob sie cluster ueberlappen!
        overlapp = check_overlapp(cluster, ex, ey)
        cluster, csys, ret = fill_zeros_random(cluster, ex, ey, clustershape) # bring into right clustershape
        if ret == True and overlapp == True:
            arr_cluster[i] = cluster
            c_sys[i] = csys
        else:
            ind_i.append(i)
            '''
            plt.imshow(cluster, norm=LogNorm())
            plt.colorbar()
            plt.show()
            '''
            if overlapp == False:
                num_overlapp = num_notoverlapp + 1
    t2 = time.time()
    print("This took ", t2-t1, "s")
    print(len(ind_i), " clusters need to be cut")
    print(num_notoverlapp, " clusters didn't overlapp")
    return arr_cluster, c_sys, np.array(ind_i)

In [12]:
num = 0
end = 1000
cluster, csys, ind_i = form_cluster(xMC[num:end], yMC[num:end], EMC[num:end], clustershape=(7,7))

This took  0.25081920623779297 s
15  clusters need to be cut
0  clusters didn't overlapp


Ok let's work on an other problem first!

In [13]:
def prep_trainingsdata(x_truth, y_truth, E_truth, coordin):
    # retruns [x rel 1, y rel 1, E1, x rel 2, y rel 2, E2]
    training = np.array([x_truth.T[0]-coordin.T[0], y_truth.T[0]-coordin.T[1], E_truth[0], x_truth.T[1]-coordin.T[0], y_truth.T[1]-coordin.T[1], E_truth[1]]).T
    print("Prepared 'training' data")

Ok all good, now lets deal with the coral data set...

In [29]:
import array

In [56]:
def get_coral2fits(x_fit, y_fit, E_fit, num_fit):
    arr = np.ones((len(x_fit), 2))*(-1000)
    good_fit = np.where(num_fit==2)
    arr[good_fit] = x_fit[good_fit]
    return arr

In [77]:
rootfile = uproot.open("./stage4_clusters.root")
event = rootfile["user202302;1"]

In [78]:
num_fit = event["num_fit"].array(library="np") #[:40000]
x_fit = event["x_fit"].array(library="np") #[:40000]
y_fit = event["y_fit"].array(library="np") #[:40000]
E_fit = event["E_fit"].array(library="np") #[:40000]

In [65]:
print(x_fit)

[array([-5.60155643]) array([-4.41711916,  7.09554801])
 array([-39.2507458]) ... array([61.35626799, 75.85294157])
 array([-39.55297433, -46.70291101, -41.37130128])
 array([-17.48499165, -22.66511402])]


In [57]:
split = 0
x_fit_new = get_coral2fits(x_fit, y_fit, E_fit, num_fit)

ValueError: shape mismatch: value array of shape (510755,) could not be broadcast to indexing result of shape (510755,2)

In [60]:
print(x_fit[:20])

[-1000 array([-4.41711916,  7.09554801]) -1000
 array([18.97693022, 37.75414665]) -1000
 array([-159.92214351, -164.98657374]) -1000
 array([-71.36201857, -52.6358013 ]) array([-71.52123563, -67.22303311])
 array([-45.5876266 , -44.20102229]) array([-56.8906959 , -48.99968943])
 array([ 96.60925813, 112.64333482]) array([-60.9923549, -53.421783 ])
 -1000 array([-35.91299022, -41.4311644 ])
 array([-80.2785908 , -81.57564891]) array([45.17376927, 36.06048039])
 array([11.7074311 , 15.80910185]) -1000
 array([-33.52318898, -50.8271679 ])]


In [43]:
a = np.array([0,1,2,3])

In [45]:
np.insert(a, 0, np.array([1,1]))

array([1, 1, 0, 1, 2, 3])

In [50]:
myarr = np.ones((1000, 2))*(-1000)
ind = np.array([2,3,4,5,22,33,45,56,66])

Ok I give in, lets do a for loop

In [76]:
def deal_with_coral_fit_format(arr, num_fit):
    nice_arr = np.ones((len(arr), 2))*(-1000)
    good_ind = np.where(num_fit==2)
    for i in good_ind[0]:
        nice_arr[i] = arr[i]
    return nice_arr

In [83]:
new_x  = deal_with_coral_fit_format(x_fit, num_fit)
print(np.all(new_x ==x_fit_new))

True


In [73]:
x_fit_new = deal_with_coral_fit_format(x_fit)
y_fit_new = deal_with_coral_fit_format(y_fit)
E_fit_new = deal_with_coral_fit_format(E_fit)

In [74]:
print(x_fit_new[:10])

[[-1000.         -1000.        ]
 [   -4.41711916     7.09554801]
 [-1000.         -1000.        ]
 [   18.97693022    37.75414665]
 [-1000.         -1000.        ]
 [ -159.92214351  -164.98657374]
 [-1000.         -1000.        ]
 [  -71.36201857   -52.6358013 ]
 [  -71.52123563   -67.22303311]
 [  -45.5876266    -44.20102229]]


In [75]:
print(y_fit_new[:10])

[[-1.00000000e+03 -1.00000000e+03]
 [ 6.48587688e+01  6.71942715e+01]
 [-1.00000000e+03 -1.00000000e+03]
 [ 1.58843447e+01  1.62712656e+01]
 [-1.00000000e+03 -1.00000000e+03]
 [-4.96006683e+00  1.30293588e+01]
 [-1.00000000e+03 -1.00000000e+03]
 [-8.00437274e+00 -6.05039581e+00]
 [ 7.67974631e-01 -5.34097408e+00]
 [ 7.03340088e+01  7.91358410e+01]]


In [84]:
print(E_fit_new[:10])

[[-1000.         -1000.        ]
 [   82.02410126    40.1187706 ]
 [-1000.         -1000.        ]
 [    3.38509297   148.07713318]
 [-1000.         -1000.        ]
 [   10.66936779    90.32144928]
 [-1000.         -1000.        ]
 [   33.2181282    122.9189682 ]
 [  145.49751282    11.81332111]
 [   38.70973969    12.16453457]]


In [85]:
print(E_fit_new.shape)

(709731, 2)


In [88]:
print(E_truth)

[array([4.27098142e-02, 1.17207406e+02]) array([77.91576198, 38.54147611])
 array([126.89557947,  69.4624554 ]) ... array([83.84694766, 25.5193812 ])
 array([ 2.46579317, 20.17808011]) array([53.91883228, 26.39064237])]


In [89]:
E_truth = np.array(event["E_truth"].array()) 

In [90]:
print(E_truth.shape)

(709731, 2)
